# Creating and modelling metallic supercells

In this section we will be concerned with modelling supercells of aluminium.
When dealing with periodic problems there is no unique definition of the
lattice: Clearly any duplication of the lattice along an axis is also a valid
repetitive unit to describe exactly the same system.
This is exactly what a **supercell** is: An $n$-fold repetition along one of the
axes of the original lattice.

The following code achieves this for aluminium:

In [1]:
using DFTK
using LinearAlgebra
using ASEconvert

function aluminium_setup(repeat=1; Ecut=7.0, kgrid=[2, 2, 2])
    a = 7.65339
    lattice = a * Matrix(I, 3, 3)
    Al = ElementPsp(:Al, psp=load_psp("hgh/lda/al-q3"))
    atoms     = [Al, Al, Al, Al]
    positions = [[0.0, 0.0, 0.0], [0.0, 0.5, 0.5], [0.5, 0.0, 0.5], [0.5, 0.5, 0.0]]
    unit_cell = periodic_system(lattice, atoms, positions)

    # Make supercell in ASE:
    # We convert our lattice to the conventions used in ASE, make the supercell
    # and then convert back ...
    supercell_ase = convert_ase(unit_cell) * pytuple((repeat, 1, 1))
    supercell     = pyconvert(AbstractSystem, supercell_ase)

    # Unfortunately right now the conversion to ASE drops the pseudopotential information,
    # so we need to reattach it:
    supercell = attach_psp(supercell, Al="hgh/lda/al-q3")

    # Construct an LDA model and discretise
    # Note: We disable symmetries explicitly here. Otherwise the problem sizes
    #       we are able to run on the CI are too simple to observe the numerical
    #       instabilities we want to trigger here.
    model = model_LDA(supercell; temperature=1e-3, symmetries=false)
    PlaneWaveBasis(model; Ecut, kgrid)
end;

As part of the code we are using a routine inside the ASE,
the [atomistic simulation environment](https://wiki.fysik.dtu.dk/ase/index.html)
for creating the supercell and make use of the two-way interoperability of
DFTK and ASE. For more details on this aspect see the documentation
on Input and output formats.

Write an example supercell structure to a file to plot it:

In [2]:
setup = aluminium_setup(5)
convert_ase(periodic_system(setup.model)).write("al_supercell.png")

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144


Python None

<img src="https://docs.dftk.org/stable/examples/al_supercell.png" width=500 height=500 />

As we will see in this notebook the modelling of a system generally becomes
harder if the system becomes larger.

- This sounds like a trivial statement as *per se* the cost per SCF step increases
  as the system (and thus $N$) gets larger.
- But there is more to it:
  If one is not careful also the *number of SCF iterations* increases
  as the system gets larger.
- The aim of a proper computational treatment of such supercells is therefore
  to ensure that the **number of SCF iterations remains constant** when the
  system size increases.

For achieving the latter DFTK by default employs the `LdosMixing`
preconditioner [^HL2021] during the SCF iterations. This mixing approach is
completely parameter free, but still automatically adapts to the treated
system in order to efficiently prevent charge sloshing. As a result,
modelling aluminium slabs indeed takes roughly the same number of SCF iterations
irrespective of the supercell size:

[^HL2021]:
   M. F. Herbst and A. Levitt.
   *Black-box inhomogeneous preconditioning for self-consistent field iterations in density functional theory.*
   J. Phys. Cond. Matt *33* 085503 (2021). [ArXiv:2009.01665](https://arxiv.org/abs/2009.01665)

In [3]:
self_consistent_field(aluminium_setup(1); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298539002742                   -0.85    5.1         
  2   -8.300209028363       -2.78       -1.26    1.5    120ms
  3   -8.300433365966       -3.65       -1.89    2.4    134ms
  4   -8.300462320222       -4.54       -2.75    2.4    146ms
  5   -8.300464320785       -5.70       -3.12    2.4    190ms
  6   -8.300464478051       -6.80       -3.29    1.9    128ms
  7   -8.300464555926       -7.11       -3.43    1.8    129ms
  8   -8.300464598290       -7.37       -3.56    1.9    132ms
  9   -8.300464630665       -7.49       -3.76    1.9    137ms
 10   -8.300464636812       -8.21       -3.87   10.9    252ms
 11   -8.300464642899       -8.22       -4.12    3.5    214ms


In [4]:
self_consistent_field(aluminium_setup(2); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -16.67464902590                   -0.70    6.5         
  2   -16.67874079534       -2.39       -1.14    1.8    302ms
  3   -16.67920994736       -3.33       -1.87    2.5    326ms
  4   -16.67927648211       -4.18       -2.74    3.4    397ms
  5   -16.67928585470       -5.03       -3.18    4.6    456ms
  6   -16.67928618915       -6.48       -3.47    2.2    345ms
  7   -16.67928621806       -7.54       -3.92    2.1    350ms
  8   -16.67928622388       -8.24       -4.57    2.5    361ms


In [5]:
self_consistent_field(aluminium_setup(4); tol=1e-4);

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.29878546915                   -0.56    6.5         
  2   -33.32384271262       -1.60       -1.00    1.5    999ms
  3   -33.33525144749       -1.94       -1.65    5.4    1.24s
  4   -33.33612474112       -3.06       -2.50    7.4    1.36s
  5   -33.33683381355       -3.15       -2.31    7.8    1.68s
  6   -33.33686844738       -4.46       -2.40    3.1    1.14s
  7   -33.33694368796       -4.12       -3.62    1.8    1.20s
  8   -33.33694390471       -6.66       -3.90    5.8    1.70s
  9   -33.33694392524       -7.69       -4.36    1.9    1.10s


When switching off explicitly the `LdosMixing`, by selecting `mixing=SimpleMixing()`,
the performance of number of required SCF steps starts to increase as we increase
the size of the modelled problem:

In [6]:
self_consistent_field(aluminium_setup(1); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -8.298582858226                   -0.85    4.9         
  2   -8.300282728814       -2.77       -1.59    1.5   99.1ms
  3   -8.300427647090       -3.84       -2.54    2.6    120ms
  4   -8.300380917576   +   -4.33       -2.29    3.4    157ms
  5   -8.300463855476       -4.08       -3.29    1.1   94.8ms
  6   -8.300464566950       -6.15       -3.69    2.2    122ms
  7   -8.300464637175       -7.15       -4.17    1.4   98.2ms


In [7]:
self_consistent_field(aluminium_setup(4); tol=1e-4, mixing=SimpleMixing());

┌ Warning: Skipping atomic property pseudopotential, which is not supported in ASE.
└ @ ASEconvert ~/.julia/packages/ASEconvert/XF0Hn/src/ASEconvert.jl:144
n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -33.32658706536                   -0.56    7.4         
  2   -33.30961289141   +   -1.77       -1.26    1.1    836ms
  3   -9.459698623360   +    1.38       -0.38    6.4    2.11s
  4   -33.32662637426        1.38       -1.74    5.4    2.00s
  5   -33.00706278086   +   -0.50       -1.20    4.1    1.51s
  6   -30.34174473898   +    0.43       -0.82    4.4    1.50s
  7   -33.33173948040        0.48       -2.01    4.4    1.41s
  8   -33.33378837996       -2.69       -2.14    2.4    1.10s
  9   -33.33622092750       -2.61       -2.39    2.2    1.08s
 10   -33.33616812581   +   -4.28       -2.60    1.8    999ms
 11   -33.33679325518       -3.20       -2.91    2.2    978ms
 12   -33.33689635397       -3.99      

For completion let us note that the more traditional `mixing=KerkerMixing()`
approach would also help in this particular setting to obtain a constant
number of SCF iterations for an increasing system size (try it!). In contrast
to `LdosMixing`, however, `KerkerMixing` is only suitable to model bulk metallic
system (like the case we are considering here). When modelling metallic surfaces
or mixtures of metals and insulators, `KerkerMixing` fails, while `LdosMixing`
still works well. See the Modelling a gallium arsenide surface example
or [^HL2021] for details. Due to the general applicability of `LdosMixing` this
method is the default mixing approach in DFTK.